# Scenario 1 - Passive Manager: Choosing Initial Squad
## Import required libraries

In [ ]:
import numpy
import pandas
from fuzzywuzzy import process
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from rsome import ro
from rsome import grb_solver

## 1. Data Cleaning
### Import data

In [ ]:
epl20_21_df = pandas.read_csv(filepath_or_buffer='./raw_data/scenario_1/EPL_20_21.csv')
players_raw_list = [pandas.read_csv(filepath_or_buffer='./raw_data/scenario_1/players_raw_' + str(i) + '_' + str(i + 1) + '.csv') for i in range(16, 21)]

# Filter columns
for i in range(len(players_raw_list)):
    players_raw_list[i] = players_raw_list[i][['first_name', 'second_name', 'team', 'total_points']]

# Read in only required columns
fifa_df_list = [pandas.read_csv(filepath_or_buffer='./raw_data/scenario_1/players_21.csv', usecols=[3, 9, 16]), pandas.read_csv(filepath_or_buffer='./raw_data/scenario_1/FIFA21_official_data.csv', usecols=[1, 8, 20]), pandas.read_csv(filepath_or_buffer='./raw_data/scenario_1/fifa21_male2.csv', usecols=[1, 5, 7])]
fifa_df_list[0]

### Rename columns to standardise

In [ ]:
fifa_df_list[0].rename(columns={'long_name': 'Name'}, inplace=True)
fifa_df_list[0].rename(columns={'club_name': 'Club'}, inplace=True)
fifa_df_list[0].rename(columns={'player_positions': 'Position'}, inplace=True)

fifa_df_list[2].rename(columns={'BP': 'Position'}, inplace=True)
fifa_df_list[0]

### Clean EPL data

In [ ]:
epl20_21_df.loc[:, 'Position'] = epl20_21_df['Position'].str.split(pat=',', n=1).str[0]
epl20_21_df

### Filter premier league teams
These data are from the game FIFA, and it will be used to supplement missing values.<br>As such, it will be wise to filter the data to only include the premier league teams.

In [ ]:
pl_team_list = ['Arsenal', 'Aston Villa', 'Brentford', 'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leicester City', 'Liverpool', 'Manchester City', 'Manchester United', 'Newcastle United', 'Norwich City', 'Sheffield United', 'Southampton', 'Tottenham Hotspur', 'Watford', 'West Ham United', 'Wolverhampton Wanderers']

# Filter teams
for i in range(len(fifa_df_list)):
    fifa_df_list[i] = fifa_df_list[i][fifa_df_list[i]['Club'].isin(values=pl_team_list)]
    fifa_df_list[i] = fifa_df_list[i].drop(columns=['Club'])
fifa_df_list[0]

### Clean FIFA data

In [ ]:
forwards = ['RW', 'LW', 'CF', 'ST', 'LS', 'RS']
midfielders = ['RCM', 'CAM', 'LCM', 'RDM', 'CDM', 'LM', 'LDM', 'RM', 'CM']
defenders = ['CB', 'RB', 'LB', 'RWB', 'LWB', 'RCB', 'LCB']



fifa_df_list[0].loc[:, 'Position'] = fifa_df_list[0]['Position'].str.split(pat=',', n=1).str[0]
fifa_df_list[1].loc[:, 'Position'] = fifa_df_list[1]['Position'].str.split(pat='>', n=2).str[1]

# Set sub and reserves to null
fifa_df_list[1]['Position'].replace(to_replace='SUB', value=numpy.nan, inplace=True)
fifa_df_list[1]['Position'].replace(to_replace='RES', value=numpy.nan, inplace=True)

# Clean position
for i in range(len(fifa_df_list)):
    fifa_df_list[i].loc[fifa_df_list[i]['Position'].isin(values=forwards), 'Position'] = 'FW'
    fifa_df_list[i].loc[fifa_df_list[i]['Position'].isin(values=midfielders), 'Position'] = 'MF'
    fifa_df_list[i].loc[fifa_df_list[i]['Position'].isin(values=defenders), 'Position'] = 'DF'
fifa_df_list[0]

### Define helper functions

In [ ]:
def standardise_na(dataframe: pandas.DataFrame) -> None:
    """
    Standardise NaN values in a dataframe.
    :param dataframe: Dataframe to standardise NaN values in.
    """
    possible_nas = ['None', 'NA', 'Nan', 'NAN', 'NaN', 'nan', 'na', 'N/A', 'none', 'null']

    for column in dataframe.columns:
        dataframe.loc[dataframe[column].astype(dtype=str).isin(values=possible_nas), column] = numpy.nan

def combine_names(dataframe: pandas.DataFrame) -> None:
    """
    Combine first and last name into one column.
    :param dataframe: Dataframe to combine names in.
    """
    dataframe['full_name'] = dataframe['first_name'] + ' ' + dataframe['second_name']
    dataframe.drop(columns=['first_name', 'second_name'], inplace=True)

def merge_df(df1: pandas.DataFrame, df2: pandas.DataFrame) -> pandas.DataFrame:
    """
    Merge two dataframes. Will only keep columns in df1.
    :param df1: Main dataframe.
    :param df2: Secondary dataframe.
    :return: Combined dataframe.
    """
    combined = df1.merge(right=df2, how='left', left_on='full_name', right_on='full_name', suffixes=('', '_'))

    for column in df1.columns:
       if column in df2.columns and (df1[column].dtype == 'int64' or df1[column].dtype == 'float64') and column != 'team':
          combined[column] = combined[[column, column + '_']].mean(numeric_only=True, axis=1)

    return combined[df1.columns]

### Clean Nan values & combine names

In [ ]:
for df in players_raw_list:
    standardise_na(dataframe=df)
    combine_names(dataframe=df)

players_raw_list[0]

### Merge all dataframes into one

In [ ]:
merged_df = players_raw_list[0]

for i in range(1, len(players_raw_list) - 1):
    merged_df = merge_df(df1=players_raw_list[i], df2=merged_df)

merged_df

### Merge on different dataset to fill positions

In [ ]:
merged_df = merged_df.merge(right=epl20_21_df[['Name', 'Position']], how='left', left_on='full_name', right_on='Name')
merged_df

### Perform fuzzy matching on names and fill in matched names

In [ ]:
def fuzzy_match(df1: pandas.DataFrame, df2: pandas.DataFrame, threshold: int) -> pandas.DataFrame:
    """
    Fuzzy match names on two dataframes.
    Reference: https://towardsdatascience.com/fuzzywuzzy-basica-and-merging-datasets-on-names-with-different-transcriptions-e2bb6e179fbf
    :param df1: dataframe 1.
    :param df2: dataframe 2.
    :param threshold: threshold for fuzzy matching.
    """
    # Get rows with missing names
    null_row_list = df1[df1['Name'].isna()]

    # Match names
    keys = {}
    for name in null_row_list['full_name']:
        temp = (process.extract(query=name, choices=df2['Name'], limit=1))
        if temp[0][1] > threshold:
            keys[name] = temp[0][0]

    # Fill names
    df1['Name'].fillna(value=df1['full_name'].map(keys), inplace=True)

    # Merge datasets
    df1 = df1.merge(right=df2, how='left', left_on='Name', right_on='Name', suffixes=('', '_'))

    # Fill missing values and cLean up columns
    df1['Position'].fillna(value=df1['Position_'], inplace=True)
    df1.drop(columns=['Position_'], inplace=True)

    return df1

In [ ]:
for i in range(90, 60, -10):
    for df in fifa_df_list:
        merged_df = fuzzy_match(df1=merged_df, df2=df, threshold=i)

merged_df.dropna(subset=['Position'], inplace=True)

### Create new club column based on team

In [ ]:
pl_dict = dict(zip([i for i in range(1, 21)], pl_team_list))
merged_df['Club'] = merged_df['team'].map(pl_dict)

### Clean up columns

In [ ]:
merged_df.drop(columns=['Name', 'team'], inplace=True)
merged_df.rename(columns={'total_points': 'Total Points', 'full_name': 'Name'}, inplace=True)

In [ ]:
merged_df.drop_duplicates(subset=['Name'], inplace=True)

### Export to csv

In [ ]:
merged_df[['Total Points', 'Name', 'Position', 'Club']].to_csv(path_or_buf='./clean_data/scenario_1/dataset.csv', index=False)

### Prepare and export costs data for year to model for (2021/2022)

In [ ]:
latest_data = pandas.read_csv(filepath_or_buffer='./raw_data/scenario_1/players_raw_21_22.csv')
costs_data = latest_data[['now_cost', 'first_name', 'second_name']]
combine_names(dataframe=costs_data)
costs_data.rename(columns={'now_cost': 'Cost', 'full_name': 'Name'}, inplace=True)
costs_data.to_csv(path_or_buf='./clean_data/scenario_1/costs_dataset.csv', index=False)

In [ ]:
evaluator_data = latest_data[['total_points', 'first_name', 'second_name']]
combine_names(dataframe=evaluator_data)
evaluator_data.rename(columns={'total_points': 'Total Points', 'full_name': 'Name'}, inplace=True)
evaluator_data.to_csv(path_or_buf='./clean_data/scenario_1/evaluator_dataset.csv', index=False)

## 2. Build Model to Solve Initial Squad Problem
### Read datasets

In [ ]:
dataset = pandas.read_csv(filepath_or_buffer='./clean_data/scenario_1/dataset.csv')
dataset

In [ ]:
costs_dataset = pandas.read_csv(filepath_or_buffer='./clean_data/scenario_1/costs_dataset.csv')
costs_dataset

In [ ]:
evaluator_dataset = pandas.read_csv(filepath_or_buffer='./clean_data/scenario_1/evaluator_dataset.csv')
evaluator_dataset

### Declare function

In [ ]:
def onehot_encode(data: pandas.Series) -> (numpy.ndarray, LabelEncoder):
    """
    One-hot encodes data, returns 2D array and encoder that 
    can be used to reverse encoding to retrieve original data.
    Reference: https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
    :param data: 1D data with categorical values.
    :returns encoded: 2D encoded values.
    :returns label_encoder: Encoder, can be used to reverse encoding to retrieve original values.
    
    To reverse encoding: label_encoder.inverse_transform([argmax(encoded[0, :])])
    """
    label_encoder = LabelEncoder()
    int_encoded = label_encoder.fit_transform(y=data)
    int_encoded = int_encoded.reshape(len(int_encoded), 1)

    onehot_encoder = OneHotEncoder(sparse=False)
    encoded = onehot_encoder.fit_transform(X=int_encoded)
    return encoded, label_encoder

### Create model to select squad

In [ ]:
def merge_df_on_name(dataframe: pandas.DataFrame, costs_df: pandas.DataFrame) -> pandas.DataFrame:
    """
    Prepares data for model. Merges historical df with current season's costs.
    Takes intersection of keys.
    :param dataframe: Dataframe with all players.
    :param costs_df: Dataframe with player names and current costs.
    :return: Merged Dataframe.
    """
    return dataframe.merge(right=costs_df, how='inner', on='Name')
    
def solve_model(dataframe: pandas.DataFrame, budget: int) -> (numpy.ndarray, float):
    """
    Solves model given data and budget.
    :param dataframe: Dataframe with all players.
    :param budget: Budget to use.
    :return: Results from solved model.
    """    
    x = dataframe['Total Points'].to_numpy()
    p = dataframe['Cost'].to_numpy()
    (pos_matrix, _) = onehot_encode(data=dataframe['Position'])
    (t, _) = onehot_encode(data=dataframe['Club'])
    max_players_per_team = 3
    max_players = 15

    # Position requirement: DF, FW, GK, MF
    position_req = numpy.array([5, 3, 2, 5])

    model = ro.Model(name='Initial Squad')

    # Define binary decision variables - players to choose
    y = model.dvar(shape=len(dataframe), vtype='B')

    model.max(y @ x)

    model.st(y @ p <= budget)
    model.st(y.sum() <= max_players)
    model.st(y @ t <= max_players_per_team)
    model.st(y @ pos_matrix == position_req)

    model.solve(solver=grb_solver, display=False)

    return y.get(), model.get()

### Solve model

In [ ]:
# budget is in 100,000
price_budget = 1000

dataset = merge_df_on_name(dataframe=dataset, costs_df=costs_dataset)
solved_y, solved_model = solve_model(dataframe=dataset, budget=price_budget)
dataset[solved_y == 1][['Name', 'Position', 'Club', 'Cost', 'Total Points']]

In [ ]:
dataset[solved_y == 1].sum(numeric_only=True)

### Evaluate Models against Current Season

In [ ]:
print("Points earned by chosen lineup in 2021/2022 season:", dataset[solved_y == 1].merge(right=evaluator_data, how='left', left_on="Name", right_on="Name")['Total Points_y'].sum())

### Export initial squad for Scenario 2

In [ ]:
dataset[solved_y == 1].to_csv('./clean_data/scenario_2/dataset.csv', index=False)

## 3. Sensitivity Analysis

### Change the budget to see the effect on points

In [ ]:
analysis_df = pandas.DataFrame(columns=['Budget', 'Total Points'], dtype=float)

for var_budget in range(650, 1650, 50):
    solved_y, solved_model = solve_model(dataframe=dataset, budget=var_budget)
    temp_df = pandas.DataFrame(data=[[var_budget, sum(dataset['Total Points'][solved_y == 1])]], columns=['Budget', 'Total Points'])
    analysis_df = pandas.concat(objs=[analysis_df, temp_df])
analysis_df.plot(x='Budget', y='Total Points', kind='line', figsize=(20, 10), title='Total Points vs Budget (in 100,000)')